## Libraries

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from pathlib import Path

In [13]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [14]:
from sklearn.svm import SVR

## Load data

In [63]:
data = pd.read_csv("..//bike_sharing_day.csv")

In [64]:
# data.dteday = pd.to_datetime(data.dteday)
data.dteday = pd.DatetimeIndex(data.dteday).month

In [65]:
data.dteday

0       1
1       1
2       1
3       1
4       1
       ..
726    12
727    12
728    12
729    12
730    12
Name: dteday, Length: 731, dtype: int64

In [17]:
# boston = load_boston()

# # Initializing the dataframe
# data = pd.DataFrame(boston.data)

# #Adding the feature names to the dataframe
# data.columns = boston.feature_names
# data.head()

In [18]:
#Adding target variable to dataframe
# data['PRICE'] = boston.target 

In [32]:
# Spliting target variable and independent variables
X = data.drop(['cnt'], axis = 1)
y = data['cnt']

In [33]:
# Matrix with feature transformation
# X = pd.read_csv("..//newBike_sharing_day.csv")

In [34]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [35]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>)

## Optimization function

In [78]:
def opt_bas(C, epsilon, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    metric = mean_absolute_error(y_test, y_pred)
    
    return -metric

## Run optimization

In [79]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [80]:
# Bounded region of parameter space
pbounds = {'C': (100, 200), 'epsilon': (0.02, 0.15), 'gamma': (0.05, 0.7)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston_mae_leo.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [81]:
optimizer.maximize(init_points=6, n_iter=500)